# IBMi + Jupyter Notebooks Template

Using Jupyter cell magic to execute DB2 for i queries

### Configure IBMi Hosts

In [1]:
import pyodbc
import pandas as pd
from ipywidgets import widgets, Text, HBox
from IPython.display import display
from IPython.core.magic import register_cell_magic

# disable pandas df display limit, trust user to limit query results
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

w_host_list = widgets.Text(placeholder='Enter hosts (comma delimited)', description='Hosts: ')
display(w_host_list)

Text(value='', description='Hosts: ', placeholder='Enter hosts (comma delimited)')

### Enter Credentials

In [2]:
host_list = [h for h in w_host_list.value.split(',') if len(h.strip()) > 0]

if len(host_list) > 0:
    w_host = widgets.Dropdown(options=host_list, value=host_list[0], description='Host: ')
else:
    w_host = widgets.Text(placeholder='Enter host', description='Host: ')
w_user = widgets.Text(placeholder='Enter user', description='User: ')
w_pwd = widgets.Password(placeholder='Enter password', description='Password: ')

display(HBox((w_host, w_user, w_pwd)))

### Notebook setup
(You shouldn't have to edit this)

In [3]:
host,user,pwd = w_host.value.strip(), w_user.value.strip(), w_pwd.value
conn = pyodbc.connect(driver='{IBM i Access ODBC Driver}', system=host, uid=user, pwd=pwd)
print("Successfully connected to host '{}' as user '{}'".format(host, user))

# Add IBMi DB2 SQL magic: invoke using %%db2i
@register_cell_magic
def db2i(line, cell):
    sql = ' '.join([x.replace('{','{{').replace('}','}}').replace('\n','').strip() for x in cell.split('\n')])
    if len(sql) > 0:
        sql = sql.format(**globals())
        if line.strip() != '-':
            return pd.read_sql(sql, conn)
        else:
            conn.execute(sql)
    else:
        print('Cannot execute blank query')
del db2i

Successfully connected to host 'DEV400' as user 'OTTEB'


In [4]:
%%db2i 

select TABLE_SCHEMA, TABLE_NAME, TABLE_PARTITION, SOURCE_TYPE
from QSYS2.SYSPARTITIONSTAT
where TABLE_SCHEMA='BOLIB'
  and TABLE_NAME='QRPGLESRC'
order by TABLE_PARTITION
limit 25

,TABLE_SCHEMA,TABLE_NAME,TABLE_PARTITION,SOURCE_TYPE
0,BOLIB,QRPGLESRC,BTCHASH,RPGLE
1,BOLIB,QRPGLESRC,CLR019B,SQLRPGLE
2,BOLIB,QRPGLESRC,FIZZBUZZ,RPGLE
3,BOLIB,QRPGLESRC,HXCVT,RPGLE
4,BOLIB,QRPGLESRC,PGMCALL,RPGLE
5,BOLIB,QRPGLESRC,POPADDRGEO,SQLRPGLE
6,BOLIB,QRPGLESRC,RPGTESTING,SQLRPGLE
7,BOLIB,QRPGLESRC,SHA256,RPGLE
8,BOLIB,QRPGLESRC,TESTPOPUP,RPGLE


In [5]:
%%db2i

select 
    id, 
    username, 
    url, 
    (minutes / 60) as hours_watched
from json_table(
    Systools.HttpPostClob(
        'https://graphql.anilist.co',
        cast('<httpHeader><header name="Content-Type" value="application/json"/></httpHeader>' as clob),
        cast('{"query": "{User(search:\"barrettotte\"){id name siteUrl stats{watchedTime}}}"}' as clob)
    ),
    '$.data.User'
    columns(
        id       char(10) path '$.id',
        username char(20) path '$.name',
        url      char(32) path '$.siteUrl',
        minutes  char(10) path '$.stats.watchedTime'
    )
)

,ID,USERNAME,URL,HOURS_WATCHED
0,247578,barrettotte,https://anilist.co/user/247578,1639
